In [2]:
import pygwalker as pyg
import pandas as pd

In [3]:
# Carregando os dados
compra_itens = pd.read_csv('/Users/FVMN/Documents/GitHub/upciga_streamlit/csv/compra_itens.csv', delimiter=';')
produtos = pd.read_csv('/Users/FVMN/Documents/GitHub/upciga_streamlit/csv/produto.csv', delimiter=';', encoding='latin-1')
compras = pd.read_csv('/Users/FVMN/Documents/GitHub/upciga_streamlit/csv/compra.csv', sep=';', encoding='latin1')
fornecedores = pd.read_csv('/Users/FVMN/Documents/GitHub/upciga_streamlit/csv/fornecedores.csv', sep=';', encoding='latin1')
vendas=pd.read_csv('/Users/FVMN/Documents/GitHub/upciga_streamlit/csv/venda.csv', delimiter=';', encoding='latin-1')
venda_itens = pd.read_csv('/Users/FVMN/Documents/GitHub/upciga_streamlit/csv/venda_itens.csv', delimiter=';', encoding='latin1')
clientes=pd.read_csv('/Users/FVMN/Documents/GitHub/upciga_streamlit/csv/clientes.csv', delimiter=';', encoding='latin-1')

# Mesclando as tabelas vendas e clientes
venda_cliente = pd.merge(vendas[['FLOJA','IDPEDIDO','IDCLI','SI_CAIXA','FDATAEMI','HORA','FVALVENDA']],clientes[['IDFORN','NOMEFORN']],how='left',left_on='IDCLI',right_on='IDFORN')
venda_cliente.rename(columns= {'NOMEFORN': 'NOMECLI'},inplace=True)
venda_cliente.drop('IDFORN', axis='columns')

# Substituindo NaN por Consumidor Final
venda_cliente['NOMECLI'].fillna('Consumidor Final', inplace=True)


In [4]:
# Criando tabela dos produtos que mais venderam
produto_mais_venda = venda_itens.groupby('PCOD')['VLVENDA'].sum().reset_index()

# Localizando os produtos que mais venderam
produto_mais_venda = produto_mais_venda.sort_values(by='VLVENDA', ascending=False)

# Selecionando os top 5 produtos que mais venderam
top_produtos_mais_vendidos = produto_mais_venda.head(5)

# Agora mesclando para saber o nome dos produtos
verifica_nome_produtos_vendas = top_produtos_mais_vendidos.merge(produtos[['PCOD', 'PDESC', 'PUNIDADE', 'PVLUVENDA' , 'PVLUVEN3']], on='PCOD')


In [5]:
verifica_nome_produtos_vendas

,PCOD,VLVENDA,PDESC,PUNIDADE,PVLUVENDA,PVLUVEN3
0,4870,94488.66,ESCORA METALICA 31O SEM CANECA,PC,143.70,87.66
1,4108,39600.00,ESCORA METALICA 31O CM MD L,PC,227.30,118.07
2,291,18560.32,GIRICA CH14 REF ROLAMENTO/ ESF,UNID,1556.00,554.39
3,5042,17451.96,FORCADO DUPLO P/ TORRE,PC,116.01,58.90
4,375,11946.00,VIGA METALICA V8 2.00 MT,PC,274.19,110.41


In [6]:
clientes_que_mais_comprou = venda_cliente.groupby('IDCLI')['FVALVENDA'].sum().reset_index()
clientes_que_mais_comprou = clientes_que_mais_comprou.sort_values(by='FVALVENDA', ascending=False)
clientes_que_mais_comprou



,IDCLI,FVALVENDA
15,831,142082.30
4,206,50588.40
12,740,14000.00
25,1337,12360.96
49,1772,12133.33
...,...,...
54,1792,260.00
23,1311,141.00
22,1209,108.00
56,1794,84.00


In [18]:
# Agrupando e somando os valores de vendas para cada cliente
clientes_que_mais_comprou = venda_cliente.groupby('IDCLI')['FVALVENDA'].sum().reset_index()

# Ordenando os clientes pelo valor total de vendas em ordem decrescente
clientes_que_mais_comprou = clientes_que_mais_comprou.sort_values(by='FVALVENDA', ascending=False)

# Realizando o merge com a tabela de clientes
tabela_clientes_mais_compraram = pd.merge(clientes_que_mais_comprou[['IDCLI', 'FVALVENDA']],clientes[['IDFORN', 'NOMEFORN', 'UF']],left_on='IDCLI', right_on='IDFORN', how='left')
tabela_clientes_mais_compraram = tabela_clientes_mais_compraram.dropna(subset=['NOMEFORN'])
tabela_clientes_mais_compraram.drop(['IDFORN'], axis=1)


tabela_clientes_mais_compraram = tabela_clientes_mais_compraram.sort_values(by='FVALVENDA', ascending=False)

alteracao_colunas = ['IDCLI','NOMEFORN','UF','FVALVENDA',]
tabela_clientes_mais_compraram = tabela_clientes_mais_compraram.reindex(columns=alteracao_colunas)
tabela_clientes_mais_compraram


,IDCLI,NOMEFORN,UF,FVALVENDA
0,831,SHOPPING DOS CO,PB,142082.30
1,206,JAILSON FRETES,SP,50588.40
2,740,CALANGO EXPRESS,PE,14000.00
9,446,BOSIO MADEIRAS,PB,5315.60
11,969,MARCONDES CARVA,PB,4500.00
16,214,JULIO CESAR FER,PB,4000.00
21,777,FRETE DE MARCON,PB,2750.00
26,905,BORRACHARIA PAI,PB,1669.20
34,516,POSTO OPCAO JD,PB,1057.34
35,1171,FM DO BRASIL,SP,900.00


In [8]:
fornecedor_mais_compra = compras.groupby('IDFORN')['FVALCOMPRA'].sum().reset_index()

fornecedor_mais_compra = fornecedor_mais_compra.sort_values(by='FVALCOMPRA', ascending=False)

nome_fornecedor_mais_compra = fornecedor_mais_compra.merge(fornecedores,on='IDFORN')
nome_fornecedor_mais_compra = nome_fornecedor_mais_compra.reindex(columns=['IDFORN','NOMEFORN','FVALCOMPRA','UF','CP_CIDADE'])

nome_fornecedor_mais_compra.iloc[0]['NOMEFORN']

'RICARDO FARDAS'

In [9]:

data_compra_fornecedor = compras.groupby('FDATAEMI')['FVALCOMPRA'].sum().reset_index()

data_compra_fornecedor = data_compra_fornecedor.sort_values(by='FDATAEMI',ascending=True)
data_compra_fornecedor['FDATAEMI'] = pd.to_datetime(data_compra_fornecedor['FDATAEMI'], format='%m/%d/%Y')
data_compra_fornecedor['FDATAEMI'] = data_compra_fornecedor['FDATAEMI'].dt.strftime('%d/%m/%Y')
data_compra_fornecedor

,FDATAEMI,FVALCOMPRA
0,02/08/2023,46572.75
1,03/08/2023,998.00
2,04/08/2023,6125.70
3,07/08/2023,3304.79
4,10/08/2023,12272.39
5,11/08/2023,29291.67
6,15/08/2023,23248.34
7,16/08/2023,771.19
8,17/08/2023,370.27
9,21/08/2023,109357.88


In [16]:
# Calcular a soma da coluna 'FVALCOMPRA' e converter para float
soma_compra_xml = compras['FVALCOMPRA'].sum()

# Formatar a soma como uma string de moeda (Real)
soma_compra_xml = f'R$ {soma_compra_xml:,.2f}'

# Exibir o valor formatado
print(soma_compra_xml)

R$ 311,223.49
